In [ ]:
%pip install langchain_community groq faiss-cpu sentence-transformers pypdf langchain_huggingface



[notice] A new release of pip available: 22.2.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [61]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.documents import Document

loader = PyPDFLoader("doc.pdf")
docs = loader.load()

len(docs)  

26

In [62]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
chunks = text_splitter.split_documents(docs)
len(chunks)  

56

In [63]:
documents = chunks
documents

[Document(metadata={'producer': 'Aspose.PDF for .NET 19.8', 'creator': 'Aspose Ltd.', 'creationdate': '', 'moddate': '2023-04-06T10:11:19-04:00', 'source': 'doc.pdf', 'total_pages': 26, 'page': 0, 'page_label': '1'}, page_content='UNITED STATES DISTRICT COURT \nNORTHERN DISTRICT OF TEXAS \nAMARILLO DIVISION \n \nALLIANCE FOR HIPPOCRATIC \nMEDICINE, et al., \n  \n   \n Plaintiffs,   \n   \n v.  Case No. 2:22-cv-00223-Z \n   \nU.S. FOOD AND DRUG \nADMINISTRATION, et al., \n  \n   \n Defendants. \n \n  \n \n \nAMICUS CURIAE BRIEF OF MISSISSIPPI, ALABAMA, ALASKA, \nARKANSAS, FLORIDA, GEORGIA, IDAHO, INDIANA, IOWA, KANSAS, \nKENTUCKY, LOUISIANA, MONTANA, NEBRASKA, OHIO, OKLAHOMA, \nSOUTH CAROLINA, SOUTH DAKOTA, TENNESSEE, TEXAS, UTAH, AND \nWYOMING IN SUPPORT OF PLAINTIFFS’ MOTION  \nFOR PRELIMINARY INJUNCTION \n \n \n \nCase 2:22-cv-00223-Z   Document 100   Filed 02/13/23    Page 1 of 26   PageID 3760'),
 Document(metadata={'producer': 'Aspose.PDF for .NET 19.8', 'creator': 'Aspose Ltd.', 

In [64]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
embeddings = HuggingFaceEmbeddings()

db = FAISS.from_documents(documents, embeddings)



In [65]:
db

In [66]:
query="Amici curiae are the States of Mississippi, Alabama, Alaska "
result=db.similarity_search(query)
result[0].page_content

'1 \n \nINTRODUCTION, INTEREST OF AMICI CURIAE,  \nAND SUMMARY OF ARGUMENT \nLast year, the Supreme Court held that abortion is a matter that is entrusted \nto “the people and their elected representatives” to address. Dobbs v. Jackson \nWomen’s Health Organization , 142 S. Ct. 2228, 2284 (2022). Overruling precedent \nthat took that auth ority away from the people, the Court returned the issue of \n“regulating or prohibiting abortion” to “the citizens of each State.” Ibid.  States may \nthus pursue their “legitimate interests” in protecting unborn life, women’s health, \nand the medical profession’s integrity by regulating or restricting abortion. Ibid. \nAmici curiae are the States of Mississippi, Alabama, Alaska, Arkansas, \nFlorida, Georgia, Idaho, Indiana, Iowa, Kansas, Kentucky, Louisiana, Montana, \nNebraska, Ohio, Oklahoma, South Carolina, South Dakota, Tennessee, Texas, Utah, \nand Wyoming. Like other States, amici have, consistent with the Constitution and'

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
You are a legal analyst. Using ONLY the provided context, produce EXACTLY 10 pivotal items for the case.
For EACH item, provide:
- A concise title
- Arguments FOR
- Arguments AGAINST
- Precise citation(s) with page numbers from the context
Rules:
- Cite only pages/lines present in the retrieved context.
- Do not invent facts, if a side is implied, say so briefly and cite the closest support.

<context>
{context}
</context>
Question: {input}""")

In [75]:
retriever=db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000284C2569780>, search_kwargs={})

In [ ]:
from groq import Groq
import os
os.environ["GROQ_API_KEY"] = "GROQ API KEY"
client = Groq(api_key=os.environ.get("GROQ_API_KEY"))
print("Groq client initialized successfully!")


Groq client initialized successfully!


In [77]:
from langchain_core.language_models.llms import LLM
from langchain_core.callbacks.manager import CallbackManagerForLLMRun
from typing import Any, List, Optional
import json

class GroqLLM(LLM):
    """Custom Groq LLM wrapper for LangChain"""
    
    model_name: str = "llama-3.3-70b-versatile"
    temperature: float = 0.1
    max_tokens: int = 2048
    
    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:
        """Call the Groq API"""
        try:
            response = client.chat.completions.create(
                model=self.model_name,
                messages=[{"role": "user", "content": prompt}],
                temperature=self.temperature,
                max_tokens=self.max_tokens,
                stop=stop,
            )
            return response.choices[0].message.content
        except Exception as e:
            return f"Error calling Groq API: {str(e)}"
    
    @property
    def _llm_type(self) -> str:
        return "groq"


llm = GroqLLM(model_name="llama-3.3-70b-versatile", temperature=0.1)
print("Groq LLM initialized successfully!")


Groq LLM initialized successfully!


In [71]:
# Test query to verify LLM is working
test_response = llm("What is the capital of France?")
print("Test response:", test_response)


Test response: The capital of France is Paris.


In [78]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

# document chain with Groq LLM
document_chain = create_stuff_documents_chain(llm, prompt)
print("Document chain created with Groq LLM!")

# retrieval chain with Groq LLM
retrieval_chain = create_retrieval_chain(retriever, document_chain)
print("Retrieval chain created with Groq LLM!")


Document chain created with Groq LLM!
Retrieval chain created with Groq LLM!


In [79]:
response = retrieval_chain.invoke({"input":"identify the top 10 key items that are pivotal to the case. These items must encompass both the arguments for and against, providing a balanced perspective that is crucial for strategic planning."})
print("Groq-powered response:")
print(response['answer'])


Groq-powered response:
Here are the top 10 pivotal items for the case:

1. **Federal Authority Over FDA Actions**
	* Arguments FOR: The FDA's actions are subject to federal authority and oversight (Page 6).
	* Arguments AGAINST: Limited information is provided on the specific federal authority in question (No direct citation).
	* Citation: Page 6 of 26, PageID 3765

2. **State Attorneys General Involvement**
	* Arguments FOR: Multiple state Attorneys General are involved, indicating a strong state interest (Pages 6-7).
	* Arguments AGAINST: The involvement of state Attorneys General may not be sufficient to establish standing (No direct citation).
	* Citation: Pages 6-7 of 26, PageID 3765

3. **Public Interest and Equities**
	* Arguments FOR: The public interest and equities support relief against the FDA's actions (Page 24).
	* Arguments AGAINST: Limited information is provided on the specific public interest and equities in question (No direct citation).
	* Citation: Page 24 of 26, P